In [5]:
from sklearn.datasets import fetch_openml
import numpy as np

# Load MNIST dataset using scikit-learn
mnist = fetch_openml('mnist_784')

# Extract features (images) and labels
images, labels = mnist.data, mnist.target.astype(np.uint8)

from sklearn.model_selection import train_test_split

# Split data to train(80%) and test
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


### Another method using tensorflow
# from tensorflow.keras import datasets
# (train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Display number of samples for train and test
print(len(train_images), len(test_images))

C:\Users\omar\AppData\Roaming\Python\Python311\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


56000 14000
